# Forte Tutorial 1.01: Running Forte in Jupyter notebooks 

---

In this tutorial we are going to explore how to interact with Forte in Jupyter notebooks using the Python API.

## Import Python modules
The first step necessary to interact with Forte is to import the `psi4` and `forte` Python modules:

In [1]:
import psi4
import forte

## Preparing Forte object with the `ObjectsUtilPsi4` module

A common first task when interacting with the Forte API is to compute 1- and 2-electron integrals in the MO basis. Here we will use the `ObjectsUtilPsi4` module to generate a `ForteData` object that contains integrals and other useful information.

At this stage, we can specify how the orbital should be partitioned or dropped off from a computation.
Here we specify the list of frozen doubly occupied orbitals (`FROZEN_DOCC`) and pass it as a dictionary to the `ObjectsUtilPsi4` module.

In [2]:
# setup xyz geometry
molecule = psi4.geometry("""
O
H 1 1.0
H 1 1.0 2 180.00
""")

# freeze the oxygen 1s orbital. The rest of the orbitals will be active
mos_spaces = {'FROZEN_DOCC' : [1,0,0,0,0,0,0,0]}
data = forte.modules.ObjectsUtilPsi4(molecule=molecule, basis="sto-3g", mo_spaces=mos_spaces).run()

# SCF Energy = -74.84159216002936



  Preparing forte objects from a Psi4 Wavefunction object   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on MacBook-Pro-506.attlocal.net
*** at Sat Jan 11 15:35:59 2025

   => Loading Basis Set <=

    Name: STO-3G
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line    81 file /Users/fevange/Source/psi4/objdir-Release/stage/share/psi4/basis/sto-3g.gbs 
    atoms 2-3 entry H          line    19 file /Users/fevange/Source/psi4/objdir-Release/stage/share/psi4/basis/sto-3g.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Co

## Molecular orbital spaces
Information about the molecular orbitals (number, symmetry, dimension of spaces, etc.) is store in the `MOSpaceInfo` class. We can easily access this object from the `ForteData` object:

In [12]:
mo_space_info = data.mo_space_info

# Grab the number of irreps
nirrep = mo_space_info.nirrep()
print("Number of irreps: ", nirrep)

# Grab the number of orbitals in each irrep
nmopi = mo_space_info.dimension("ALL").to_tuple()
print("Number of orbitals in each irrep: ", nmopi)

# Grab the number of active orbitals in each irrep
nactpi = mo_space_info.dimension("ACTIVE").to_tuple()
print("Number of active orbitals in each irrep: ", nactpi)

# Grab the total number of active orbitals
nact = mo_space_info.size("ACTIVE")
print("Total number of active orbitals: ", nact)

# Grab the symmetry of each orbital
symm = mo_space_info.symmetry("ACTIVE")
print("Symmetry of each the active orbitals: ", symm)

# Grab the symmetry labels
irrep_labels = mo_space_info.irrep_labels()
print("Irrep labels: ", irrep_labels)

Number of irreps:  8
Number of orbitals in each irrep:  (3, 0, 0, 0, 0, 2, 1, 1)
Number of active orbitals in each irrep:  (2, 0, 0, 0, 0, 2, 1, 1)
Total number of active orbitals:  6
Symmetry of each the active orbitals:  [0, 0, 5, 5, 6, 7]
Irrep labels:  ['Ag', 'B1g', 'B2g', 'B3g', 'Au', 'B1u', 'B2u', 'B3u']


> **Task 1**: Take a look at the file `forte/src/base_classes/mo_space_info.h`. This class stores information about molecule orbital space. However, only one function is exposed. Create a github/forte pull request to expose functions that you need to find out the number of orbital in each space (including symmetry).

## The `ForteIntegral` object

In Forte there are two classes responsible for handling integrals:
- `ForteIntegral`: reads the integrals from Psi4 and stores them in varios formats (conventional, density fitting, Cholesky, ...).
- `ActiveSpaceIntegrals`: stores a copy of all the integrals in the active space (used by active space methods). This class only stores a subset of the integrals and includes an effective potential due to non-active doubly occupied orbitals.

We will first access the `ForteIntegral` object:

In [14]:
ints = data.ints
print(f'Number of molecular orbitals: {ints.nmo()}')
print(f'Number of correlated molecular orbitals: {ints.ncmo()}')

Number of molecular orbitals: 7
Number of correlated molecular orbitals: 6


> **Task 2**: Take a look at the file `forte/src/integrals/integrals.h`. This class allows the user to access the 1-/2-electron integrals via functions like `double oei_a(size_t p, size_t q)`. However, these functions are not exposed to the python side. Expose one of these functions and commit them to github/forte.


## The `ActiveSpaceIntegrals` object

The `ActiveSpaceIntegrals` object is a convenient container for storing the integrals in the active space.

In [16]:
as_ints = data.as_ints

print(f'Nuclear repulsion energy = {as_ints.nuclear_repulsion_energy()}')
print(f'Frozen-core energy = {as_ints.frozen_core_energy()}')
print(f'Scalar energy = {as_ints.scalar_energy()}') # any additional energy terms

Nuclear repulsion energy = 8.731423976054998
Frozen-core energy = -60.56668027959451
Scalar energy = 0.0


The frozen-core energy is the energy of the electrons in the frozen doubly occupied orbitals. The scalar energy is a contribution that comes from the restricted doubly occupied orbitals. This quantity arises because in the `ActiveSpaceIntegrals` object we only consider the active orbitals and the restricted doubly occupied levels contribute with a scalar shift to the energy (the scalar energy) and they also modify the one-electron integrals. We call these effecti one-electron integrals ($\langle\phi_p|\hat{h}'|\phi_q\rangle$)

We can also access individual elements of the 1-/2-electron integrals. This class stores five types of integrals:

- the alpha effective one-electron integrals, $\langle\phi_p|\hat{h}'|\phi_q\rangle$, via the `oei_a` function.
- the beta effective one-electron integrals, $\langle\phi_{\bar{p}}|\hat{h}'|\phi_{\bar{q}}\rangle$, via the `oei_b` function.
- the alpha-alpha antisymmetrized two-electron integrals, $\langle\phi_p \phi_q\|\phi_r\phi_s\rangle$, via the `tei_aa` function.
- the alpha-beta two-electron integrals, $\langle\phi_p \phi_{\bar{q}}\|\phi_r\phi_{\bar{s}}\rangle = \langle\phi_p \phi_{\bar{q}}|\phi_r\phi_{\bar{s}}\rangle$, via the `tei_ab` function
- the beta-beta two-electron integrals, $\langle\phi_{\bar{p}}\phi_{\bar{q}}\|\phi_{\bar{r}}\phi_{\bar{s}}\rangle$, via the `tei_bb` function

Here we denote beta spin orbitals with a bar above the orbital index.

In [19]:
print(f'<0a|h|0a>    = {as_ints.oei_a(0,0)}')
print(f'<0a0a||0a0a> = {as_ints.tei_aa(0,0,0,0)}')
print(f'<0a0b||0a0b> = {as_ints.tei_ab(0,0,0,0)}')

<0a|h|0a>    = -5.584167676739151
<0a0a||0a0a> = 0.0
<0a0b||0a0b> = 0.6953370997674169


We can also print all the integrals at once (see the `output.dat` file)

In [20]:
as_ints.print()



  ==> Active Space Integrals <==

  Nuclear repulsion energy:         8.731423976055
  Frozen core energy:             -60.566680279595
  Scalar energy:                    0.000000000000

One-electron integrals (alpha) <p|h|q> (includes restricted docc)
  < 0|h| 0> =      -5.584167676739
  < 0|h| 1> =      -0.815811210385
  < 1|h| 0> =      -0.815811210385
  < 1|h| 1> =      -3.469940807960
  < 2|h| 2> =      -4.662234465580
  < 2|h| 3> =      -1.143829056262
  < 3|h| 2> =      -1.143829056262
  < 3|h| 3> =      -3.886974727464
  < 4|h| 4> =      -5.217566642281
  < 5|h| 5> =      -5.217566642281

One-electron integrals (beta) <p|h|q> (includes restricted docc)
  < 0|h| 0> =      -5.584167676739
  < 0|h| 1> =      -0.815811210385
  < 1|h| 0> =      -0.815811210385
  < 1|h| 1> =      -3.469940807960
  < 2|h| 2> =      -4.662234465580
  < 2|h| 3> =      -1.143829056262
  < 3|h| 2> =      -1.143829056262
  < 3|h| 3> =      -3.886974727464
  < 4|h| 4> =      -5.217566642281
  < 5|h| 5> =